# Squirrels In My Park (SIMP)
<img src="candace-squirrel.gif" width="1000" >

### Group Information
    
Mikayla Compton, mikayla.compton@utah.edu, u1538218  <br> 
Ashley Pritchard u1256025@utah.edu, U1256025 <br>
Maya Wheeler, maya.wheeler@hsc.utah.edu, u0999875 <br>
Repository Link: https://github.com/pritchardashley/SIMP<br>
Topic: Understanding the correlation between physical squirrel attributes and location on behavior

### Background and Motivation


Almost everywhere we go, campus, a park, or even a big city, there will be squirrels. There are over 280 species of squirrels around the world (4) and come in a variety of colors: Black, brown, grey, red, and even containing purple, orange, and maroon (1) Research has found that squirrels have personalities and similar behaviors to humans (3). There has also been some research into the patterns of dominance in squirrels with older squirrels being more aggressive than younger squirrels and similar patterns for larger and smaller squirrels (6). There is a lot more to learn about squirrels and how and why they behave the ways that they do.


A lot of our interest on this topic stems from our personal observations, specifically with the squirrels on the University of Utah campus. We as a group concluded that the level of human interactions, and food availability may have impacted their behaviors in certain situations. The squirrels on the University of Utah campus even have their own portion of the University of Utah website dedicated to them (2) as well as an annual “Squirrel Fest” held by the Natural History Museum (5). Although the data set of squirrels that we will be using is not based in Utah, we feel as though we can learn a lot of information, through the application of skills learned in class, to gain a better understanding of the squirrels in Utah and around the world. 


### Project Objectives


The primary question that we want to try to answer within our project is if there is a strong correlation between the physical characteristics and location of the squirrels in central park and their observed behaviors. The specific characteristics that we are focusing on are fur color, highlighted fur color, age, and location of the squirrels. In a part of this we want to see if the squirrels that show fear towards interactions with humans have different characteristics than those squirrels that do not. Our null hypothesis is that the physical characteristics have no effect on behavioral observations and our alternative hypothesis is that the physical characteristics have an effect on the observed behaviors. 	

    
Our learning objective is to utilize what we have learned in class to perform an accurate analysis of our data. Potential skills we hope to apply and understand more deeply include but are not limited to: R-squared, P-value, OLS Regression, descriptive statistics, categorical data, dummy values, data clean, and data visualization. Another thing we want to look into is creating a map of the squirrels in Central Park that shows our results for better visualization.


### Data Description and Acquisition


Our data comes from the 2018 NYC Squirrel Census. This was a citizen science project that had volunteers observing squirrels in Central Park in New York City. The data is in two CSV files, as well as a word document with data definitions. The first file has our primary data, which includes 3022 observations. Each observation has 7 columns identifying the location, time, date, etc for the observation. There are 23 different columns with boolean or nominal values for different attributes of the observed squirrel, such as color (grey, cinnamon or black), what the squirrel was doing(eating, foraging, etc.), and behaviors it displayed, including vocalizations or interactions with humans. The second file is a CSV containing the latitude and longitude of the hectares that the park was broken into for the observations. 


Squirrel census data:
https://data.cityofnewyork.us/Environment/2018-Central-Park-Squirrel-Census-Squirrel-Data/vfnx-vebw/about_data


Hectare data:
https://data.cityofnewyork.us/dataset/2018-Central-Park-Squirrel-Census-Hectare-Grid/qad5-y26n/about_data


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from scipy.stats import chisquare
import folium
import re
from shapely.geometry import Point, Polygon
import shapely
import geopandas as gpd
from shapely.wkt import loads
from shapely import wkt

In [ ]:
#importing our data
nyc_squirrels=pd.read_csv('2018_Central_Park_Squirrel_Census_-_Squirrel_Data_20250304.csv')
hectare = pd.read_csv('2018_Central_Park_Squirrel_Census_-_Hectare_Grid_20250304.csv')

### Ethical Considerations


The group that would be the most affected by our analysis would be the squirrel community within central park. Depending on what we find in our analysis it could encourage people to interact more with squirrels. This, in turn, could lead to the endangerment of the squirrels or an increase of stress onto them. If people are influenced to interact with the squirrels it could also lead to the transmission of disease between the squirrels and humans. With a possible influx of people interacting with squirrels this would also mean a direct increase of people to central park. This could affect other aspects of the park outside of the squirrel population like people straying off paths or stepping on plants. 


### Data Cleaning and Processing

After completing the first milestone, we realized the data cleaning and processing would be signifiantly more complex than originally anticipated.<br>
The census data was fairly complete. The variables with the most missing values are not ones we intend to use, so those columns were dropped. The remaining rows with missing values were dropped as well.

In [ ]:
#making a missing values heat map
plt.figure(figsize=(8,6))
sns.heatmap(nyc_squirrels.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Values Heatmap')
#plt.savefig('Missing Squirrel Data', dpi=300)
plt.show()

In [ ]:
#check amount of missing data in each column 
missing_data = nyc_squirrels.isna().sum()
print("Total squirrels in dataset: ", len(nyc_squirrels))
#drop unneeded columns 
drop_columns = ['Color notes', 'Specific Location', 'Other Activities', 'Other Interactions', 'Highlight Fur Color']
clean_squirrels = nyc_squirrels.drop(columns = drop_columns)
clean_squirrels = clean_squirrels.dropna()
missing_data_ap = clean_squirrels.isna().sum()
print(missing_data_ap)

This is where the processing got more complicated. The bulk of the cleaning for this dataset was in the hectare data. The hectare dataset listed the latitude and longitude in a single string with extensive formatting, so they needed to be extracted, using a regular expression, and split into seperate columns.

In [ ]:
# Extract coordinate pairs using regex
def extract_coords(wkt):
    # Remove the leading "MULTIPOLYGON (((" and trailing ")))"
    cleaned = re.sub(r'^MULTIPOLYGON\s+\(\(\(|\)\)\)$', '', wkt)
    # Split into individual coordinate pairs
    coord_pairs = cleaned.split(', ')
    # Split each pair into lon and lat
    coords = [coord.split() for coord in coord_pairs]
    return coords

# Apply extraction
hectare['coords'] = hectare['the_geom'].apply(extract_coords)

# Convert coordinate lists to individual columns
# Flatten coordinates into new DataFrame columns
coord_df = hectare['coords'].apply(lambda x: pd.Series([float(num) for pair in x for num in pair]))

# Rename the columns as lon_1, lat_1, lon_2, lat_2, ...
coord_df.columns = [f"{'lon' if i % 2 == 0 else 'lat'}_{i//2 + 1}" for i in coord_df.columns]

# Concatenate with original DataFrame
hectare_final = pd.concat([hectare.drop(columns='coords'), coord_df], axis=1)

The squirrel data also used a different labeling system to identify hectares than the hectare data sets, so the two labeling systems had to be unified. This was achived by taking the location of one squirrel reported in each hecatre in the squirrel dataset, and finding which hectare in the hecatre dataset it's location fell within. These labels were then paired, and the datasets joined on the corresponding hectare labels.

In [ ]:
#saving the location of one squirrel identified in each hectare
squirrel_hec_id = nyc_squirrels[['X','Y','Hectare']]
squirrel_hec_id = squirrel_hec_id.sort_values(by = 'Hectare')
squirrel_hec_id.drop_duplicates(subset=["Hectare"], keep="first", inplace=True)
squirrel_hec_id.reset_index(drop=True, inplace=True)

#making and naming the polygons with shapely
hectares_shapely = []
for i in range(len(hectare_final)):
    p = (Polygon([(hectare_final.iloc[i,7],hectare_final.iloc[i,6]),
                  (hectare_final.iloc[i,9],hectare_final.iloc[i,8]),
                  (hectare_final.iloc[i,11],hectare_final.iloc[i,10]),
                  (hectare_final.iloc[i,13],hectare_final.iloc[i,12])]),
                 f'{hectare_final.iloc[i,1]}')
    hectares_shapely.append(p)

#finding which hectare each squirrel point belongs in
hectare_assign = []
for i in range(len(squirrel_hec_id)):
    p = Point(squirrel_hec_id.iloc[i,1],squirrel_hec_id.iloc[i,0])
    for s in hectares_shapely:
        if s[0].contains(p) == True:
            h = (([squirrel_hec_id.iloc[i,2],s[1]]))
            hectare_assign.append(h)
     
hectare_assign = pd.DataFrame(hectare_assign, columns = ['Hectare', 'id'])
hectare_assign['id'] = hectare_assign['id'].astype('int64')

#joinging the squirrels data with the hectare id number
nyc_squirrels = clean_squirrels.merge(hectare_assign, how='left', on='Hectare')
nyc_squirrels

We wanted to look deeper into different areas of the park and displays of fear. We separated the hectares of the park into hectares that are at the perimeter and hectares that are "internal" as seen below, so we could test if proximity to a major road impacted fear behaviors.

In [ ]:
#Internal hectares and perimeter hectares

#Convert to shapely geometries
hectare['geometry'] = hectare['the_geom'].apply(loads)

#Make a GeoDataFrame from the regular DataFrame
hectare_gdf = gpd.GeoDataFrame(hectare, geometry='geometry')

#Set a coordinate reference system
hectare_gdf.set_crs("EPSG:4326", inplace=True)

#Find outer limits of the grid
min_x = hectare_gdf['xmin'].min()
max_x = hectare_gdf['xmax'].max()
min_y = hectare_gdf['ymin'].min()
max_y = hectare_gdf['ymax'].max()

#Mark perimeter: anything that touches the bounding box of the grid
hectare_gdf['is_perimeter'] = (
    (hectare_gdf['xmin'] == min_x) | (hectare_gdf['xmax'] == max_x) |
    (hectare_gdf['ymin'] == min_y) | (hectare_gdf['ymax'] == max_y)
)

#Mark interior as not perimeter
hectare_gdf['is_interior'] = ~hectare_gdf['is_perimeter']

print(hectare_gdf[['id', 'is_interior', 'is_perimeter']])

#Plot
fig, ax = plt.subplots(figsize=(10, 10))
hectare_gdf[hectare_gdf['is_interior']].plot(ax=ax, color='green', edgecolor='black', label='Interior')
hectare_gdf[hectare_gdf['is_perimeter']].plot(ax=ax, color='red', edgecolor='black', label='Perimeter')
plt.title("Interior vs Perimeter Hectares")
plt.axis('off') 
plt.show()

#add interior and perimeter columns to the dataframe
in_out_hectare = pd.DataFrame(hectare_gdf, columns = ['is_interior', 'is_perimeter', 'id'])
in_out_hectare['id'] = in_out_hectare['id'].astype('int64')

nyc_squirrels = nyc_squirrels.merge(in_out_hectare, how='left', on='id')
nyc_squirrels

Finally, we wanted a way to catagorize the squirrels as having displayed a behavior associated with fear or not. There are several behavior columns that correlate to fear, so we created an additional column to aggregate these behaviors. We decided that a squirrel showed fear if the observer noted at least one of the following: Quaas, moans, tail flags, or running from something as squirrels will generally do one of the following in the precense of a predator in order to either scare them away or confuse them.

In [ ]:
#Dispays of fear
fear_columns = ['Quaas', 'Moans', 'Tail flags', 'Runs from']
fear = nyc_squirrels[fear_columns].apply(pd.Series.value_counts)
print(fear)

#add a new column where it is marked true if it showed at least one fear trait
nyc_squirrels["showed_fear"] = nyc_squirrels[fear_columns].map(
    lambda x: str(x).strip().lower() in ["yes", "true", "1"]
).any(axis=1)
nyc_squirrels

### Exploratory Analysis


The first thing we wanted to look into was the frequency of squirrel fur colors which we show in this bar chart.

In [ ]:
#Primary fur color
fur_color_ap = clean_squirrels['Primary Fur Color'].value_counts()

plt.figure(figsize=(8, 5))
fur_color_ap.plot(kind="bar", color=["Gray", "Red", "Black"])

plt.title("Frequency of Squirrel Fur Colors")
plt.xlabel("Primary Fur Color")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

#Combination of primary and highlight
prim_high_color_ap = clean_squirrels['Combination of Primary and Highlight Color'].value_counts()

plt.figure(figsize=(10, 5))
prim_high_color_ap.plot(kind="bar", color=["Gray", "indianred", "lightgray", "rosybrown", "firebrick", "mistyrose", "Black", "Maroon", "rosybrown", "darkred", "dimgray",
                                          "brown", "brown", "saddlebrown", "dimgray", "gray", "sienna", "sienna", "saddlebrown", "gainsboro", "gray"])

plt.title("Frequency of Primary and Highlight Squirrel Fur Colors")
plt.xlabel("Primary Fur Color")
plt.ylabel("Count")
plt.xticks(rotation=45, ha="right")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

Next, we wanted to take a look into the different behaviors that were classified as signs of fear to see how they compared to one another.

In [ ]:
#Dispays of fear
plt.figure(figsize=(8, 5))
fear.plot(kind="bar", color=["Blue", "lightblue", "Darkblue"])

plt.title("Frequency of Display of Fear Behaviors")
plt.xlabel("Expressing Fear Based Behavior")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

Once we looked at those freqencies we then wanted to shift gears and focus on the location of the squirrels within Central Park. None of us have worked with geographic data before and wanted to play around with it using the folium library. Once we initialized our map by finding the mean coordinates of the park we were then able to plot our points. The first map shows the squirrel locations based on primary fur color and also outlines the hectares. The map allows the user to hover over a squirrel and see the squirrel ID associated with that specific squirrel. The second map shows us the hecatres based on how many squirrels show fear within them. The green squares mean the fear rate is lower than 50% and the red squares indicate the fear rate within that hectare is greater than 50%. The second map also shows us the squirrel locations but grouped based on if they showed fear(indicated by the scared face), or if they expressed a lack of fear (indicated by the happy face).

In [ ]:
# Initialize the map
fur_colors = {
    "Gray": {
        "icon_url": "graysquirrel.png",  # Gray squirrel
        "layer": folium.FeatureGroup(name="Gray Squirrels")
    },
    "Cinnamon": {
        "icon_url": "squirrel.png",  # Cinnamon squirrel
        "layer": folium.FeatureGroup(name="Cinnamon Squirrels")
    },
    "Black": {
        "icon_url": "blacksquirrel.png",  # Black squirrel
        "layer": folium.FeatureGroup(name="Black Squirrels")
    }
}

for index, row in nyc_squirrels.iterrows():
    fur_color = row.get('Primary Fur Color', 'Unknown')

    if fur_color in fur_colors:
        icon = folium.CustomIcon(
            icon_image=fur_colors[fur_color]["icon_url"],
            icon_size=(20, 20)
        )
        folium.Marker(
            location=[row['Y'], row['X']],
            icon=icon,
            popup=row['Unique Squirrel ID']
        ).add_to(fur_colors[fur_color]["layer"])

polygon_layer = folium.FeatureGroup(name="Hectare Polygons")

for index, row in hectare_final.iterrows():
    try:
        locations = [
            (row["lat_1"], row["lon_1"]),
            (row["lat_2"], row["lon_2"]),
            (row["lat_3"], row["lon_3"]),
            (row["lat_4"], row["lon_4"]),
            (row["lat_1"], row["lon_1"])
        ]
        folium.Polygon(
            locations=locations,
            color="red" if index % 2 == 0 else "blue",
            fill=True,
            fill_color="red" if index % 2 == 0 else "blue",
            fill_opacity=0.3,
            popup=str(row.get("id", f"Hectare {index}"))
        ).add_to(polygon_layer)
    except Exception as e:
        print(f"Error in polygon {index}: {e}")

my_map_mc = folium.Map(location=[nyc_squirrels['Y'].mean(), nyc_squirrels['X'].mean()], zoom_start=15)


polygon_layer.add_to(my_map_mc)

for info in fur_colors.values():
    info["layer"].add_to(my_map_mc)

# Add toggle control
folium.LayerControl(collapsed=False).add_to(my_map_mc)

# Save and/or display
my_map_mc.save("grouped_squirrels_map.html")
my_map_mc


In [ ]:
# Group squirrels by id and calculate mean of 'showed_fear'
fear_rates = nyc_squirrels.groupby("id")["showed_fear"].mean().reset_index()
fear_rates.rename(columns={"showed_fear": "fear_rate"}, inplace=True)

df_final = hectare_final.merge(fear_rates, on="id", how="left")

df_final["fear_label"] = df_final["fear_rate"].apply(
    lambda x: "Fear" if pd.notna(x) and x >= 0.5 else "No Fear"
)


In [ ]:
# Rebuild the map from scratch to avoid duplicates or conflicts
my_map_mc = folium.Map(location=[nyc_squirrels['Y'].mean(), nyc_squirrels['X'].mean()], zoom_start=15)

# New FeatureGroup for labeled hectare polygons
fear_polygons_layer = folium.FeatureGroup(name="Hectare Fear Zones")

for index, row in df_final.iterrows():
    try:
        locations = [
            (row["lat_1"], row["lon_1"]),
            (row["lat_2"], row["lon_2"]),
            (row["lat_3"], row["lon_3"]),
            (row["lat_4"], row["lon_4"]),
            (row["lat_1"], row["lon_1"])  # Close the polygon
        ]

        # Choose color based on fear_label
        color = "red" if row["fear_label"] == "Fear" else "green"

        folium.Polygon(
            locations=locations,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.4,
            popup=f"Hectare ID: {row.get('hectare_id', index)}\nFear Level: {row['fear_label']}"
        ).add_to(fear_polygons_layer)

    except Exception as e:
        print(f"Polygon error at index {index}: {e}")

fear_polygons_layer.add_to(my_map_mc)


# fear-based icons
showed_fear_mc = {
    "Fear": {
        "icon_url": "fear.png",
        "layer": folium.FeatureGroup(name="Fear")
    },
    "No Fear": {
        "icon_url": "happiness.png",
        "layer": folium.FeatureGroup(name="No Fear")
    }
}

for index, row in clean_squirrels.iterrows():
    fear = row.get('showed_fear_ap', False)
    fear_label = "Fear" if fear else "No Fear"

    icon = folium.CustomIcon(
        icon_image=showed_fear_mc[fear_label]["icon_url"],
        icon_size=(20, 20)
    )
    folium.Marker(
        location=[row['Y'], row['X']],
        icon=icon,
        popup=row['Unique Squirrel ID']
    ).add_to(showed_fear_mc[fear_label]["layer"])

# Add fear layers to map
for info in showed_fear_mc.values():
    info["layer"].add_to(my_map_mc)

# Add the layer control toggle
folium.LayerControl(collapsed=False).add_to(my_map_mc)

# Save or display
my_map_mc.save("fear_map.html")
my_map_mc


### Analysis


We tested for the correlation (and strength of that correlation) between physical traits and behaviors, using the chi squared test of independence and Carmer’s V test. We also tested the correlation between the location in the park and squirrel behavior.


In [ ]:
#See if physical characteristics and fear have a relation

fear_columns = ['Quaas', 'Moans', 'Tail flags', 'Runs from']

#Group by fur color and count how many did/didn't show fear
fur_fear = nyc_squirrels.groupby("Primary Fur Color")["showed_fear"].value_counts().unstack()

print(nyc_squirrels["showed_fear"].value_counts())

#Plot
fur_fear.plot(kind="bar", stacked=True, figsize=(8, 5), color=["steelblue", "firebrick"])
print(fur_fear)

plt.title("Comparison of Fur Color and Fear Behaviors in Squirrels")
plt.xlabel("Fur Color")
plt.ylabel("Count of Squirrels")
plt.legend(["Did Not Show Fear", "Showed Fear"])
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()


fur_fear = nyc_squirrels.groupby("Primary Fur Color")["showed_fear"].value_counts().unstack(fill_value=0)

#contingency table
print(fur_fear)

#Chi-Square test
chi2, p, dof, expected = chi2_contingency(fur_fear)

print(f"\nChi-Square statistic: {chi2}")
print(f"Degrees of freedom: {dof}")
print(f"Expected frequencies:\n{expected}")
print(f"P-value: {p}")

From the graph and the chi-squared test, it can be observed that although there appears to be a difference in the number of squirrels that show fear based on the ratio of total squirrels of a color and the number that showed fear, the p-value indicates that any difference is likely due to chance. We cannot reject the null hypothesis that the squirrels primary fur color does not have any correlation to the demonstration of fear.

In [ ]:
#See if fear and location have a relationship

loc_fear_ap = nyc_squirrels.groupby("Lat/Long")["showed_fear"].value_counts().unstack(fill_value=0)
print(nyc_squirrels["showed_fear"].value_counts())

colors = nyc_squirrels["showed_fear"].map({True: "firebrick", False: "steelblue"})

#Plot
plt.figure(figsize=(8, 5))
plt.scatter(nyc_squirrels["X"], nyc_squirrels["Y"], c=colors, alpha=0.6)

plt.title("Squirrel Locations and Fear Response")
plt.xlabel("Longitude (X)")
plt.ylabel("Latitude (Y)")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(["Did Not Show Fear", "Showed Fear"])

plt.show()

#Chi-Square test
chi2, p, dof, expected = chi2_contingency(loc_fear_ap)

print(f"\nChi-Square statistic: {chi2}")
print(f"Degrees of freedom: {dof}")
print(f"Expected frequencies:\n{expected}")
print(f"P-value: {p}")

Each squirrel was given coordinates of their location. We wanted to look into whether thei exact location of the park has any coorelation with traits displayed. We obtained a p-value of 0.49 from a chi square test resulting in the inability to reject the null hypothesis that location plays no role in a squirrels behavioral responses.

In [ ]:
#Hectare and fear response relationship

hect_fear_ap = nyc_squirrels.groupby("Hectare")["showed_fear"].value_counts().unstack(fill_value=0)
print(nyc_squirrels["showed_fear"].value_counts())

#Plot
hect_fear_ap.plot(kind="barh", stacked=True, figsize=(15, 20), color=["steelblue", "firebrick"])

plt.title("Fear Response by Hectare")
plt.xlabel("Hectare", rotation=0)
plt.ylabel("Number of Squirrels")
plt.legend(["Did Not Show Fear", "Showed Fear"])
plt.xticks(rotation=0) 
plt.grid(axis="y", alpha=0.5)

plt.show()

#Chi-Square test
chi2, p, dof, expected = chi2_contingency(hect_fear_ap)

print(f"\nChi-Square statistic: {chi2}")
print(f"Degrees of freedom: {dof}")
print(f"Expected frequencies:\n{expected}")
print(f"P-value: {p}")

Interestingly enough, although the squirrels exact location played no stastically significant role, when comparing the squirrels in each hectare, we are able to reject the null hypothesis that the hectare tht a squirrel is in has no effect on whether it will demonstrate fear. We obtained a p-value of 9.19e-10 indicating that there is a stastical significance between hectare and squirrels experiencing fear

Because there is a statistically significant relationship between hectare and fear, we wanted to look deeper into different areas of the park and displays of fear. We separated the hectares of the park into hectares that are at the perimeter and hectares that are "internal" as seen below.

In [ ]:
#See if hectare location has a relation to the fear experienced

per_in_hec = nyc_squirrels.groupby("is_perimeter")["showed_fear"].value_counts().unstack(fill_value=0)
print(nyc_squirrels["showed_fear"].value_counts())

#Plot
per_in_hec.plot(kind="bar", stacked=True, figsize=(6, 8), color=["steelblue", "firebrick"])

plt.title("Fear Response vs Location of Hectare")
plt.xlabel("Location", rotation=0)
plt.ylabel("Number of Squirrels")
plt.legend(["Did Not Show Fear", "Showed Fear"])
plt.xticks(rotation=0) 
plt.grid(axis="y", alpha=0.5)

plt.show()

#Chi-Square test
chi2, p, dof, expected = chi2_contingency(per_in_hec)

print(f"\nChi-Square statistic: {chi2}")
print(f"Degrees of freedom: {dof}")
print(f"Expected frequencies:\n{expected}")
print(f"P-value: {p}")

Although there is a stastitcally significant difference in the hectare that a squirrel is in, whether the hectare is in the perimeter of the park or more internal does not play a role in the fear that a squirrel expresses.

Because there were multiple shifts of volunteers that made squirrel observations, we also wanted to look at whether the time of day had any effect on the squirrels behaviors. Based on the chi-squared and obtained p-value, there is not stastistically significant difference.

In [ ]:
#AM/PM and fear response relationship

time_fear_ap = nyc_squirrels.groupby("Shift")["showed_fear"].value_counts().unstack(fill_value=0)
print(nyc_squirrels["showed_fear"].value_counts())

#Plot
time_fear_ap.plot(kind="bar", stacked=True, figsize=(6, 8), color=["steelblue", "firebrick"])

plt.title("Fear Response vs Time of Day")
plt.xlabel("Time", rotation=0)
plt.ylabel("Number of Squirrels")
plt.legend(["Did Not Show Fear", "Showed Fear"])
plt.xticks(rotation=0) 
plt.grid(axis="y", alpha=0.5)

plt.show()

#Chi-Square test
chi2, p, dof, expected = chi2_contingency(time_fear_ap)

print(f"\nChi-Square statistic: {chi2}")
print(f"Degrees of freedom: {dof}")
print(f"Expected frequencies:\n{expected}")
print(f"P-value: {p}")

Research has demonstrated that adult aged squirrels tend to me more aggressive (6), however the data obtained demonstrated that there is no stastically significant difference is fear displays based on age.

In [ ]:
#age and fear response relationship

age_fear_ap = nyc_squirrels.groupby("Age")["showed_fear"].value_counts().unstack(fill_value=0)
print(nyc_squirrels["showed_fear"].value_counts())

#Plot
age_fear_ap.plot(kind="bar", stacked=True, figsize=(6, 5), color=["steelblue", "firebrick"])

plt.title("Fear Response vs Age")
plt.xlabel("Age", rotation=0)
plt.ylabel("Number of Squirrels")
plt.legend(["Did Not Show Fear", "Showed Fear"])
plt.xticks(rotation=0) 
plt.grid(axis="y", alpha=0.5)

plt.show()

#Chi-Square test
chi2, p, dof, expected = chi2_contingency(age_fear_ap)

print(f"\nChi-Square statistic: {chi2}")
print(f"Degrees of freedom: {dof}")
print(f"Expected frequencies:\n{expected}")
print(f"P-value: {p}")

The last test we did was to see if the squirrels were randomly dispersed across the park. We only included hectares with squirrels in them, as most of the empty hectares are over lakes or buildings.

In [ ]:
#making a dataframe with the necessary data
squirrel_data = nyc_squirrels[['id','Shift','Primary Fur Color','showed_fear','Age']].copy()
squirrel_data = pd.get_dummies(squirrel_data, columns=['Primary Fur Color'],prefix = 'col')
squirrel_data['showed_fear'] = squirrel_data['showed_fear'] == True
squirrel_data['no_fear'] = squirrel_data['showed_fear'] == False
color_hectare = squirrel_data.groupby('id')[['col_Black', 'col_Cinnamon', 'col_Gray']].sum()

In [ ]:
#Black Squirrel disrtibution
blk_observed_counts = color_hectare['col_Black']
# Expected counts
blk_expected_counts = [sum(blk_observed_counts) / len(blk_observed_counts)] * len(blk_observed_counts)
# Perform Chi-Square Test
blk_chi2_stat, blk_p_value = chisquare(f_obs=blk_observed_counts, f_exp=blk_expected_counts)
print("Black Squirrels")
print(f"Chi² Statistic: {blk_chi2_stat}")
print(f"P-value: {blk_p_value}")

blk_vmr = np.var(blk_observed_counts) / np.mean(blk_observed_counts)
print(f"Variance-to-Mean Ratio: {blk_vmr}")

#Cinnamon Squirrel distrtibution
cin_observed_counts = color_hectare['col_Cinnamon']
# Expected counts
cin_expected_counts = [sum(cin_observed_counts) / len(cin_observed_counts)] * len(cin_observed_counts)
# Perform Chi-Square Test
cin_chi2_stat, cin_p_value = chisquare(f_obs=cin_observed_counts, f_exp=cin_expected_counts)
print("Cinnamon Squirrels")
print(f"Chi² Statistic: {cin_chi2_stat}")
print(f"P-value: {cin_p_value}")

cin_vmr = np.var(cin_observed_counts) / np.mean(cin_observed_counts)
print(f"Variance-to-Mean Ratio: {cin_vmr}")

#Gray Squirrel distrtibution
gry_observed_counts = color_hectare['col_Gray']
# Expected counts
gry_expected_counts = [sum(gry_observed_counts) / len(gry_observed_counts)] * len(gry_observed_counts)
# Perform Chi-Square Test
gry_chi2_stat, gry_p_value = chisquare(f_obs=gry_observed_counts, f_exp=gry_expected_counts)
print(f"Chi² Statistic: {gry_chi2_stat}")
print(f"P-value: {gry_p_value}")

gry_vmr = np.var(gry_observed_counts) / np.mean(gry_observed_counts)
print(f"Variance-to-Mean Ratio: {gry_vmr}")

### Conclusion

Our project aimed to determine if there is a strong correlation between physical characteristics, including primary fur color, and age, as well as location of the squirrel within Central Park and their behaviors. While we were hopeful to find some correlation between squirrels who do show fear behavior and a common denominator to group them, our analysis did not show this. We were able to show that squirrels were not randomly distributed across the park, and were instead clustered. Through our analysis we were able to determine that different hectares squirrels are in, affects the squirrels behavior. However we were unable to determine the location of the hectares that resulted in a discrepancy of behaviors.  
An issue we frequently ran into was discrepancies within our datasets. For example, we had two datasets for the hectares, one describing the location and the other describing the conditions of that hectare, and the two datasets did not contain the same ID numbers for the same hectares so we had to manually identify each one using their coordinates. The next discrepancy we discovered was the possibility of duplicated squirrels within our dataset due to the census being collected over multiple days. Another thing we noticed within the hectare data was there were multiple volunteers inputting conflicting data regarding the conditions of the same hectare. The next issue we ran into was utilizing Github. Since we split the coding up between the three of us we were excited to be able to all work on the same jupyter notebook and then push it to our github for the other team members to see. While this worked for the first couple of pushes, once we decided to make our own branches to avoid writing over other member’s code, somewhere along the way we broke it and we were no longer able to push or pull anything.
For future analysis we would want to look more into the clusters that we found and see what factor is clustering the squirrels. We would also want to focus more on our hectare data by focusing on individual hectares so we have a smaller area to investigate. Another thing we discussed as a group was collecting better data from a different, more organized source, or potentially seeing if there is any consistency of behaviors across squirrels in different parks, or areas.


Works Cited<br>
(1) Baldwin, Marc. “Squirrel Fur Colour & Characteristics.” *Wildlife Online*, Wildlife Online, 20 Oct. 2024, www.wildlifeonline.me.uk/animals/article/squirrel-fur-colour-characteristics#:~:text=Among%20the%20Greys%2C%20silvery%20(frosted,grey%20but%20the%20feet%20orange.). <br>


(2) Fox. “Animals of the U: Fox A. Squirrel - @theU.” *AttheU*, 1 Apr. 2024, attheu.utah.edu/students/animals-of-the-u-fox-a-squirrel/. <br>


(3) Kerlin, Katherine E. “Personality Matters, Even for Squirrels.” *UC Davis*, 17 Apr. 2023, www.ucdavis.edu/curiosity/news/personality-matters-even-squirrels-0. <br>


(4) “Let’s Learn about Squirrels.” *IAISMuseums*, June 2020, www.iaismuseum.org/wp-content/uploads/2020/05/Lets-Learn-About-Squirrels.pdf. <br>


(5) “NHMU Fox Squirrel Survey.” *Natural History Museum of Utah*, nhmu.utah.edu/citizen-science/utah-fox-squirrels/survey. Accessed 17 Apr. 2025. <br>


(6) “Squirrel Behaviour - Dominance, Aggression & Territoriality.” *Wildlife Online*, Wildlife Online, 15 Sept. 2019, www.wildlifeonline.me.uk/animals/article/squirrel-behaviour-dominance-aggression-territoriality#:~:text=A%20study%20of%20Grey%20squirrel,social%20aggressiveness%20than%20older%20animals. 